# Движок бэктеста

- Симуляция торговли, расчет PL, комиссии/slippage
- Сохранение equity curve, расчет Win Rate, Sharpe, Expectancy, Profit Factor
- Валидация результатов и edge-cases

Ядро всей бэктест-логики, с подробными шагами для разных сценариев.


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

# Функция бэктест-движка
def backtest_engine(signals, returns, initial_capital=100000, commission=0.0003, slippage=0.0001):
    """
    Простой движок бэктеста
    signals: DataFrame с колонкой 'position' (1=long, -1=short, 0=no position)
    returns: Series с доходностями
    """
    portfolio = pd.DataFrame(index=signals.index)
    portfolio['position'] = signals['position']
    portfolio['returns'] = returns
    
    # Рассчитываем strategy returns
    portfolio['strategy_returns'] = portfolio['position'].shift(1) * portfolio['returns']
    
    # Вычитаем комиссию при смене позиции
    position_change = portfolio['position'].diff().abs()
    portfolio['commission_cost'] = position_change * (commission + slippage)
    portfolio['strategy_returns'] = portfolio['strategy_returns'] - portfolio['commission_cost']
    
    # Equity curve
    portfolio['equity'] = initial_capital * (1 + portfolio['strategy_returns']).cumprod()
    portfolio['buy_hold_equity'] = initial_capital * (1 + portfolio['returns']).cumprod()
    
    return portfolio


def calculate_performance_metrics(portfolio):
    """Расчет метрик производительности"""
    strategy_returns = portfolio['strategy_returns'].dropna()
    
    total_return = (portfolio['equity'].iloc[-1] / portfolio['equity'].iloc[0]) - 1
    buy_hold_return = (portfolio['buy_hold_equity'].iloc[-1] / portfolio['buy_hold_equity'].iloc[0]) - 1
    
    # Sharpe Ratio (годовой)
    sharpe = strategy_returns.mean() / strategy_returns.std() * np.sqrt(252) if strategy_returns.std() > 0 else 0
    
    # Win Rate
    winning_trades = strategy_returns[strategy_returns > 0]
    losing_trades = strategy_returns[strategy_returns < 0]
    win_rate = len(winning_trades) / len(strategy_returns) if len(strategy_returns) > 0 else 0
    
    # Expectancy
    avg_win = winning_trades.mean() if len(winning_trades) > 0 else 0
    avg_loss = losing_trades.mean() if len(losing_trades) > 0 else 0
    expectancy = (win_rate * avg_win) - ((1 - win_rate) * abs(avg_loss))
    
    # Max Drawdown
    cumulative = (1 + strategy_returns).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    max_drawdown = drawdown.min()
    
    return {
        'total_return': total_return,
        'buy_hold_return': buy_hold_return,
        'sharpe_ratio': sharpe,
        'win_rate': win_rate,
        'expectancy': expectancy,
        'max_drawdown': max_drawdown,
        'num_trades': len(strategy_returns)
    }

print("✅ Движок бэктеста загружен")


## Запуск бэктеста


In [ ]:
# Загрузка сигналов и данных
BACKTEST_DIR = Path('data') / 'backtest'
ticker = 'SBER'

signals_df = pd.read_parquet(BACKTEST_DIR / f"{ticker}_signals_channels.parquet")

# Для демонстрации создаем синтетические returns
# В реальности нужны реальные log_returns
np.random.seed(42)
returns = pd.Series(np.random.randn(len(signals_df)) * 0.02, index=signals_df.index)

# Запуск бэктеста
portfolio = backtest_engine(signals_df, returns, initial_capital=100000, commission=0.0003)

print(f"✅ Бэктест завершен")
print(f"\nEquity curve:")
print(portfolio[['equity', 'buy_hold_equity']].tail())


## Расчет метрик


In [ ]:
metrics = calculate_performance_metrics(portfolio)

print("\n=== Метрики Бэктеста ===")
print(f"Доходность стратегии: {metrics['total_return']:.2%}")
print(f"Buy & Hold доходность: {metrics['buy_hold_return']:.2%}")
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
print(f"Win Rate: {metrics['win_rate']:.2%}")
print(f"Expectancy: {metrics['expectancy']:.4f}")
print(f"Max Drawdown: {metrics['max_drawdown']:.2%}")
print(f"Число сделок: {metrics['num_trades']}")


## Сохранение результатов


In [ ]:
# Сохранение портфеля
portfolio_path = BACKTEST_DIR / f"{ticker}_backtest_portfolio.parquet"
portfolio.to_parquet(portfolio_path, index=False)

# Сохранение метрик
metrics_df = pd.DataFrame([metrics])
metrics_path = BACKTEST_DIR / f"{ticker}_backtest_metrics.csv"
metrics_df.to_csv(metrics_path, index=False)

print(f"\n✅ Результаты сохранены:")
print(f"  Portfolio: {portfolio_path}")
print(f"  Metrics: {metrics_path}")
